# AR(1)プロセスからランダムウォーク、そしてMCMCへ

このノートブックでは、時系列解析の基礎となる **AR(1)プロセス**（1次自己回帰モデル）を出発点とし、その係数 $\phi$ が 1 に近づくにつれて現れる **ランダムウォーク（酔歩）**、そしてそのランダムウォークを「探索」に応用した **メトロポリス・ヘイスティングス法 (MCMC)** までの流れを、数式とインタラクティブなシミュレーションで地続きに解説します。

---

## 1. AR(1) プロセスとは

AR(1) プロセス（AutoRegressive process of order 1）は、**「現在の値が、1つ前の値とノイズで決まる」** 最もシンプルな時系列モデルです。

$$
y_t = \phi y_{t-1} + \varepsilon_t, \quad \varepsilon_t \sim N(0, \sigma^2)
$$

ここで、
- $y_t$: 時刻 $t$ における値
- $\phi$: **自己回帰係数** (Autoregressive coefficient)
- $\varepsilon_t$: ホワイトノイズ（平均0、分散$\sigma^2$の正規分布に従う）

この $\phi$ の値によって、時系列の挙動（**定常性**）が劇的に変化します。

- **$|\phi| < 1$ (定常)**: 平均 0 の周りを振動し、長期的には安定します。
- **$|\phi| = 1$ (単位根・ランダムウォーク)**: **非定常**。平均へ回帰せず、どこまでも彷徨います。
- **$|\phi| > 1$ (発散)**: 指数関数的に値が大きくなります。

### 自己相関 (ACF) と 偏自己相関 (PACF)
ARモデルの特徴は、**コレログラム**（自己相関図）に現れます。

- **自己相関関数 (ACF)**: 時間差（ラグ）$k$ だけ離れたデータ間の相関。
    - AR(1) の場合、ACFは $\phi^k$ で減衰します（$\phi=0.8$なら $0.8, 0.64, 0.512...$）。
- **偏自己相関関数 (PACF)**: 間の時点の影響を取り除いた直接的な相関。
    - AR(1) の場合、**ラグ1 ( $\phi$ ) だけが有意な値を持ち、ラグ2以降はゼロになります**（これがARモデルの識別基準です）。

---


### 【実験】 $\phi$ を変えて挙動を確認しよう

スライダーで $\phi$ を動かして、以下の点を確認してください。
1. **$\phi = 0.5$**: ACFは速やかに減衰し、PACFはラグ1のみ立つ。
2. **$\phi = 0.9$**: ACFの減衰が遅くなる（記憶が長く残る）。
3. **$\phi = 1.0$**: **ランダムウォーク**。ACFはほとんど減衰せず、1付近に留まる（これを「単位根」と呼ぶ）。


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import ipywidgets as widgets
from IPython.display import display
import japanize_matplotlib

def simulate_ar1_interactive(phi=0.8, sigma=1.0, T=200):
    # シミュレーション
    np.random.seed(42)
    y = np.zeros(T)
    epsilon = np.random.normal(0, sigma, T)
    
    # 初期値
    if abs(phi) < 1:
        y[0] = epsilon[0] / np.sqrt(1 - phi**2)
    else:
        y[0] = 0
        
    for t in range(1, T):
        y[t] = phi * y[t-1] + epsilon[t]
        
    # プロット作成
    fig = plt.figure(figsize=(15, 5))
    
    # 1. 時系列プロット
    ax1 = fig.add_subplot(1, 3, 1)
    ax1.plot(y)
    ax1.set_title(rf"時系列 $y_t = {phi} y_{{t-1}} + \varepsilon$")
    ax1.set_xlabel("時刻 $t$")
    ax1.set_ylabel("$y_t$")
    ax1.grid(True, alpha=0.3)
    if abs(phi) >= 1:
        ax1.text(0.05, 0.9, "非定常 (Non-Stationary)", transform=ax1.transAxes, color="red", fontweight="bold")
    else:
        ax1.text(0.05, 0.9, "定常 (Stationary)", transform=ax1.transAxes, color="blue", fontweight="bold")

    # 2. ACF (自己相関)
    ax2 = fig.add_subplot(1, 3, 2)
    plot_acf(y, lags=20, ax=ax2, title="自己相関 (ACF)")
    ax2.grid(True, alpha=0.3)
    
    # 3. PACF (偏自己相関)
    ax3 = fig.add_subplot(1, 3, 3)
    try:
        plot_pacf(y, lags=20, ax=ax3, title="偏自己相関 (PACF)", method='ywm')
    except:
        ax3.text(0.5, 0.5, "計算エラー (非定常)", ha='center')
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# インタラクティブ・ウィジェット
w_phi = widgets.FloatSlider(value=0.8, min=-1.1, max=1.1, step=0.1, description=r'係数 $\phi$:', continuous_update=False)
display(widgets.interactive(simulate_ar1_interactive, phi=w_phi, sigma=widgets.fixed(1.0), T=widgets.fixed(200)))


---

## 2. 定常から非定常へ：ランダムウォーク（酔歩）

$\phi = 1$ のとき、式は以下のようになります。

$$
y_t = y_{t-1} + \varepsilon_t
$$

これは、**「昨日の位置からランダムに一歩進んだ場所が今日の位置」** というモデルです。これを **ランダムウォーク（酔歩）** と呼びます。またの名を「イカ（Squid）」の遊泳とも喩えられます（予測不可能な動き）。

### なぜこれが重要なのか？
ランダムウォークは、**「分散が時間とともに増大する」** という性質を持ちます。
$$
\mathrm{Var}(y_t) = t \sigma^2
$$
つまり、時間が経てば経つほど、**どこにいるか予測がつかなくなる（＝探索範囲が広がる）** ということです。

統計学（特にベイズ統計）では、この「どこへ行くかわからない＝空間を広く探索できる」という性質を**逆に利用**します。それが次に紹介する **MCMC（マルコフ連鎖モンテカルロ法）** です。


---

## 3. ランダムウォークの応用：メトロポリス・ヘイスティングス法

確率分布 $p(x)$ からサンプリングを行いたいとき、その分布が複雑だと直接サンプリングできません。そこで、**ランダムウォークを使って分布の「山」を探索**します。

### アルゴリズムの直感
1. **今いる場所** を $x_t$ とします。
2. 次の候補 $x_{new}$ を、**ランダムウォーク** で決めます。
   $$ x_{new} = x_t + \varepsilon, \quad \varepsilon \sim N(0, \sigma^2) $$
   （これはまさに、$\phi=1$ の AR(1) プロセスの1ステップです！）
3. $x_{new}$ が $x_t$ よりも「確率が高い（山頂に近い）」場所なら、必ず移動します。
4. 「確率が低い（麓の方）」場所なら、コイン投げをして、運が良ければ移動、悪ければ留まります。

こうすることで、ランダムウォーク（酔っ払い）は、**確率が高い場所ほど頻繁に訪れる**ようになります。その足跡（Trace）を集めると、目的の確率分布に従うサンプルが得られます。



In [ ]:
from scipy.stats import norm

def run_metropolis_hastings(n_iter=5000, step_width=1.0):
    # 目標分布: 標準正規分布 N(0, 1) とあえて少しズラした N(3, 0.5) の混合分布を想定してみる
    # シンプルにするため、ここでは標準正規分布 N(0, 1) を目標とする
    def target_pdf(x):
        return norm.pdf(x, loc=0, scale=1)

    np.random.seed(123)
    current_x = 10.0  # 初期値（あえて遠くからスタート）
    samples = []
    trace = []
    
    # 提案分布としてランダムウォークを使用
    # x_new = 1.0 * x_old + epsilon  (つまり phi=1 の AR(1))
    
    for i in range(n_iter):
        # 1. 提案 (Random Walk Step)
        epsilon = np.random.normal(0, step_width)
        proposal_x = current_x + epsilon
        
        # 2. 採択確率 (Metropolis基準)
        # p(x_new) / p(x_old)
        ratio = target_pdf(proposal_x) / target_pdf(current_x)
        acceptance_prob = min(1.0, ratio)
        
        # 3. 判定
        if np.random.rand() < acceptance_prob:
            current_x = proposal_x  # Accept (Move)
        else:
            current_x = current_x   # Reject (Stay)
            
        trace.append(current_x)
        if i > 500: # バーンイン（初期の影響を除く）
            samples.append(current_x)
            
    # 可視化
    fig = plt.figure(figsize=(15, 6))
    
    # トレースプロット（時系列）
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(trace, lw=0.8, color='navy', alpha=0.7)
    # Using raw string to avoid escape sequence issues
    ax1.set_title(rf"MCMCの軌跡 (Trace Plot)\nランダムウォーク $\phi=1$ による探索", fontsize=14)
    ax1.set_xlabel("Step $t$")
    ax1.set_ylabel("$x_t$")
    ax1.grid(True, alpha=0.3)
    ax1.text(0.05, 0.9, "探索的挙動 (Exploration)", transform=ax1.transAxes, color="green", fontweight="bold")

    # ヒストグラム（分布）
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.hist(samples, bins=50, density=True, color='skyblue', alpha=0.7, label="MCMC Samples")
    x = np.linspace(-4, 4, 100)
    ax2.plot(x, target_pdf(x), 'r-', lw=3, label="Target PDF")
    ax2.set_title("サンプリング結果の分布", fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# 実行
run_metropolis_hastings()



---
## （付録）平均の分散に関する理論的考察

まず平均を
$$
\bar{y}_T = \frac{1}{T}\sum_{t=1}^T y_t
$$
と置くと、自己共分散 $\gamma_h=\mathrm{Cov}(y_t,y_{t-h})$ を用いて
$$
\mathrm{Var}(\bar{y}_T)= \frac{1}{T^2}\sum_{t=1}^T\sum_{s=1}^T\gamma_{|t-s|}
$$
となります。AR(1)では $\gamma_h=\sigma_u^2\phi^{|h|}$ なので、二重和を整理すると
$$
\mathrm{Var}(\bar{y}_T)= \frac{1}{T^2}\Big(T\gamma_0 + 2\sum_{h=1}^{T-1}(T-h)\gamma_h\Big)
$$
$$
= \frac{1}{T^2}\Big(T\sigma_u^2 + 2\sum_{h=1}^{T-1}(T-h)\sigma_u^2\phi^h\Big)
$$
となり、相関がある（$\phi \neq 0$）と、独立な場合（iid）に比べて分散が変化することがわかります。

---


# コラム：ARIMAモデルとは

*(夜間、街の明かりに照らされた近代的な都市で、オンライン設備を見ている若いビジネスウーマン)*

### 共同執筆者
**Joshua Noble** (Data Scientist)

## ARIMAモデルの概要
ARIMAは自己回帰和分移動平均（AutoRegressive Integrated Moving Average）の略称で、時系列分析および時系列で将来生じ得る値を予測するための手法です。

自己回帰モデリングと移動平均モデリングは、時系列データを予測するための2つの異なるアプローチです。ARIMAはこれら2つのアプローチを統合しているため、この名前が付けられています。予測は、時系列の過去の動作を使用して、その時系列の1つ以上の将来の値を予測する機械学習の一分野です。小さな店でアイスクリームを買うことを想像してみてください。天候の温暖化に伴い、アイスクリームの売上が着実に増加していることをご存じであれば、おそらく来週の注文は、この数週間の注文よりも少し増えるはずだと予測できるでしょう。どの程度大きくなるかは、今週の売上が前週の売上とどれだけ異なるかによって決まります。比較する過去がなければ未来を予測することはできないため、過去の時系列データはARIMAばかりではなく、すべての予測手法や時系列分析手法にとって非常に重要です。

ARIMA は、時系列予測に最も広く使用されているアプローチの1つであり、作業する時系列データのタイプに応じて異なる2つの方法で使用できます。最初のケースでは、時系列データの季節性を考慮する必要のない非季節性ARIMAモデルを作成しました。過去のデータのパターンに基づいて未来を予測します。2番目のケースでは、時系列に影響する規則的なサイクルである季節性を考慮しています。これらのサイクルは日次、週次、月次のいずれかに指定でき、将来の値を予測するために使用できる時系列の過去データのパターンを定義するのに役立ちます。データサイエンスの多くと同様に、予測の基礎となるのは、モデルをトレーニングするための優れた時系列データを持つことです。時系列は、変数を等間隔で測定する順序付けされた一連の値です。この等間隔の時間間隔要件のため、時間要素を含むすべてのデータセットが実際には時系列データであるとは限らないことを覚えておくことが重要です。

## ボックス・ジェンキンス法
1970年、統計学者のジョージ・ボックスとグウィリム・ジェンキンスは、あらゆる種類の時系列モデルを適合させるために、ボックス・ジェンキンス法として知られるようになった方法を提案しました。このアプローチは、時系列を生成したプロセスが定常である場合、モデルを使用して近似できるという仮定から始まります。これは、次の4つのステップで構成されています。

1. **識別**：時系列が定常であるかどうかを評価し、定常でない場合は、定常とするために必要な差分の数を評価します。次に、診断プロットで使用する差分データを生成します。自己相関および部分自己相関からデータのARMAモデルのパラメーターを特定します。
2. **推定**：データを使ってモデルのパラメーター（係数）をトレーニングします。
3. **診断チェック**：利用可能なデータとの関連で適合モデルを評価し、モデルが改善される可能性のある領域をチェックします。特に、過剰適合のチェックや残差の計算が含まれます。
4. **予測**：モデルができたら、そのモデルを使用して値の予測を開始できます。

モデルがデータに正しく適合していることを確認したら、ARIMAによる予測を開始する準備が整います。これらの各ステップを詳しく見ていきます。

## 時系列データの特徴
時系列は定常時系列か非定常時系列のいずれの場合もあります。定常時系列には、時間の経過とともに一定の統計的特性があります。つまり、平均、分散、自己相関などの統計は、データによって変化しないという意味です。ARIMAなどのほとんどの統計的予測手法は、時系列が1つ以上の変換を通じてほぼ定常にできるという仮定に基づいています。定常時系列は、統計的性質が将来も過去とほぼ同じであることを単純に予測できるため、予測が比較的容易です。非固定データの操作は可能ですが、ARIMAのようなアプローチでは困難です。

時系列データの別の主な特徴は、データにトレンドが存在するかどうかです。例えば、過去50年間に食料品店で販売されていた基本食料品の価格は、インフレによって価格が上昇するため、トレンドを示すことになります。トレンドを含むデータの予測は、トレンドによってデータ内の他のパターンが不明瞭になるため、難しい場合があります。データに安定したトレンド・ラインがあり、それが一貫して回帰する場合、それはトレンド定常である可能性があります。この場合、モデルを当てはめる前に、トレンド・ラインを当てはめ、データからトレンドを差し引くだけで、トレンドを取り除くことができます。データがトレンド定常でない場合は、差分定常である可能性があり、その場合は差分化によってトレンドを取り除くことができます。差分化を行う最も簡単な方法は、各値から前の値を引いて、時系列データにどれだけの変化があるかを測定することです。例えば、$Y_t$が期間$t$における時系列Yの値である場合、期間$t$におけるYの最初の差は$Y_t - Y_{t-1}$に等しくなります。

こちらは、非定常時系列のプロットです。それは明らかな上昇傾向があり、季節性を示しています。

*(徐々に増加する時系列を示す時系列グラフ)*

ここでの季節性は、通常の12か月サイクルです。これは、時系列を12単位ずつ差分して対処すると1990年4月と1989年4月を区別できます。12単位の遅延を伴う差分を時系列に適用すると、より定常な時系列が得られます。この時系列の差異はまだ変化していますが、ARIMAモデルはこの時系列に適合し、それを使用した予測を行うことができます。

*(前のデータを差分で示し、増加傾向を除外した時系列グラフ)*

例えば、周期的な振る舞いはあるがトレンドや季節性のない時系列も定常的です。系列を観察する際にサイクルの長さが一定でない限り、サイクルのピークと谷がどこで発生するのかを知ることはできません。一般に、定常的な時系列には、長期的には予測可能なパターンはありません。時系列データを折れ線グラフにプロットすると、一定の分散があり、大きなスパイクや減少がないため、ほぼ水平方向に見えます。

## 自己相関
自己相関を計算することで、時系列が過去の値とどの程度相関しているかを知ることができます。自己相関を計算することで、データがランダム性を示しているかどうか、ある観測値がすぐ隣の観測値とどの程度関連しているかについての質問に答えることができます。これにより、どのようなモデルがデータを最もよく表しているかがわかります。自己相関は、ラグ単位までの点間の相関関係を確認するためにプロットされることがよくあります。

自己相関における各ラグ$k$は、次のように定義されます。

$$
r_k = \frac{\sum_{t=k+1}^T (y_t - \bar{y})(y_{t-k} - \bar{y})}{\sum_{t=1}^T (y_t - \bar{y})^2}
$$

$r$は自己相関の遅れ、$T$は時系列の長さ、$y$は時系列の値を表します。自己相関係数は自己相関関数（ACF）を構成します。

ACFでは、相関係数はx軸にあり、遅れの数（遅れ順序と呼ばれる）はy軸に示されます。自己相関プロットは、statsmodelsライブラリーの`plot_acf`を使用してPythonで作成でき、`acf`関数を使用してRで作成できます。

*(自己相関関数のプロット)*

このACFプロットでは、12の時間単位のラグで差分された時系列のラグは、ゼロラグ自体と完全に相関しています。最初のラグは負、2番目のラグはわずかに正、3番目のラグは負のようになります。12番目のラグはそれ自体と強く相関していることに気付くでしょう。月次データを見ていたので、これは理にかなっています。自己相関は時系列全体でほぼ同じサイクルを維持しており、時系列にはまだ大きな季節性が含まれていることを示しています。ACFプロットは、このデータに最も適合するARIMAモデルのパラメーターを推測するのにも役立ちます。



### 自己相関の直感的理解
想像してみてください。ある日の売上や気温、株価などの時系列データがあるとします。「今日の売上」と「昨日の売上」は似たような傾向があるかもしれません。もっと前の「1週間前の売上」も、何らかの関係を持っているかもしれません。自己相関とは、この「時刻差（ラグ：lag）＝k」のときに、いまの値と1つ前、2つ前……というずらし方での相関を数値で表したものです。

自己相関を調べると、「データがどのくらい過去の値に影響されているか」が見えます。
- 時系列がランダムすぎて自己相関がほとんどない場合：過去の値から未来を予測しづらい
- 自己相関が強く残っている場合：少し前のデータから未来を予想できそう
といった判断に使えます。

## 部分自己相関関数（PACF）
時系列データにARIMAモデルを使用する準備におけるもう1つの重要なプロットは、部分自己相関関数です。ACFプロットは、$y_t$と$y_{t-k}$の間の関係を、$k$のさまざまな値について示します。$y_t$と$y_{t-1}$が相関している場合、$y_{t-1}$と$y_{t-2}$も相関します。しかし、$y_{t-2}$に含まれる新しい情報が$y_t$の予測に使われるのではなく、むしろ$y_{t-1}$と$y_{t-2}$の両方が関連しているために、$y_t$と$y_{t-2}$が相関している可能性もあります。この問題を克服するためには、偏自己相関を使用して、多くのラグ観測値を取り除くことができます。これらは、ラグ1から$k$の影響を取り除いた後の$y_t$と$y_{t-k}$の関係を測定します。それで、最初の偏自己相関は最初の自己相関と同じです。それは、除去すべき何かがないからです。各部分的な自己相関は、自己回帰モデルの最後の係数として推定できます。

RやPython、その他のプログラミング言語やライブラリーで作業していても、PACFを計算し、簡単に検査できるPACFプロットを作成する方法があります。自己相関プロットは、pythonではstatsmodelsライブラリーの`plot_pacf`を使って作成でき、Rでは`pacf`関数を使って作成できます。

*(偏自己相関関数のプロット)*

このPACFは、上記のACFプロットと同じデータを使用しています。PACFプロットは、ACFプロットのように0ではなく1から始まり、前年の同じ月と相関する1.0の遅れまで強い相関を示します。その最初の1年を過ぎると、ラグの数が増えるにつれて自己相関の量が減少していきます。年ごとに変動する月次データを見ていたので、これは理にかなっています。



### 偏自己相関の「偏」とは？
偏自己相関は、自己相関のうち、間にある他の時点の影響を取り除いたうえで、どれくらい相関があるかを見るものです。
たとえば lag=3 の自己相関を見るとき、直接「3つ前のデータ」と現在のデータを比べていますが、そこには「1つ前」「2つ前」のデータの影響も含まれているかもしれません。偏自己相関は、その中間にある影響（ラグ1・ラグ2の影響）をいったん取り除いたうえで、本当に「3つ前のデータ」と「現在のデータ」だけの純粋な相関を測ります。

### ARモデルとMAモデルの見分け方
ACFとPACFのプロットの形状は、ARIMAモデルの次数（p, q）を決定する重要な手がかりになります。

- **AR(p) モデル（自己回帰モデル）の場合**:
    - **PACF**がラグ $p$ までは有意で、ラグ $p+1$ 以降は突然ゼロ付近になります（カットオフ）。
    - ACFは徐々に減衰します。
- **MA(q) モデル（移動平均モデル）の場合**:
    - **ACF**がラグ $q$ までは有意で、ラグ $q+1$ 以降は突然ゼロ付近になります（カットオフ）。
    - PACFは徐々に減衰します。

つまり、ACFとPACFのどちらが「スパッと切れる（カットオフする）」かを見比べることで、ARモデルかMAモデルかの当たりをつけることができます。

### 観点の棚卸し（AR/MA/ARIMAの識別）

| モデル | 観点1: 自己相関 (ACF) | 観点2: 偏自己相関 (PACF) |
|---|---|---|
| **AR(p) 自己回帰** | 徐々に減衰 (Tail off) | **ラグpで切断 (Cut off)** |
| **MA(q) 移動平均** | **ラグqで切断 (Cut off)** | 徐々に減衰 (Tail off) |
| **ARMA / ARIMA** | 徐々に減衰 | 徐々に減衰 |


## 自己回帰と移動平均
その名前が示すように、ARIMAという頭字語では、渡されたパラメーターに応じて、自己回帰モデルと移動平均モデルを単一のモデルに統合します。時系列全体を通じて変化をモデリングするこれら2つの方法は関連していますが、いくつかの大きな違いがあります。自己回帰モデルでは、変数の過去値の線形結合を使用して予測します。自己回帰という用語は、変数自体に対する回帰であることを示します。この手法は、過去の値を回帰のインプットとして使用する方法が線形回帰モデルと似ています。自己回帰は次のように定義されます。

$$
y_t = c + \phi_1 y_{t-1} + \phi_2 y_{t-2} + \dots + \phi_p y_{t-p} + \varepsilon_t
$$

ここでは、$\varepsilon_t$はホワイトノイズです。これは重回帰に似ていますが、予測変数として$y_t$の遅延値を使用します。これをAR(p)モデル、つまりp次の自己回帰モデルと呼びます。

一方、移動平均モデルは、回帰で予測変数の過去の値を使用するのではなく、過去の予測誤差を使用します。移動平均は、ウィンドウ内の$k$個の値を単純に平均化し、ここで、$k$は移動平均ウィンドウのサイズであり、このウィンドウを進めます。予測値は実際の値を使用して評価され、時系列の各ステップにおける誤差が特定されます。移動平均は次のように定義されます。

$$
y_t = c + \varepsilon_t + \theta_1 \varepsilon_{t-1} + \theta_2 \varepsilon_{t-2} + \dots + \theta_q \varepsilon_{t-q}
$$

$\varepsilon_t$ はホワイトノイズです。これをMA(q)モデル、つまりq次の移動平均モデルと呼びます。もちろん、 $\varepsilon_t$の値は観測しないので、通常の意味での回帰ではありません。$y_t$の各値は、過去いくつかの予測誤差の重み付けされた移動平均と考えることができることに注意してください。

通常、ARIMAモデルでは、自己回帰項（AR）項または移動平均項（MA）のいずれかを使用します。ACFプロットとPACFプロットは、これらの用語のうちどれが最も適切かを判断するためによく使用されます。

## ARIMAモデルの指定
時系列を定常化し、自己相関の性質が判別されると、ARIMAモデルを適合させることが可能になります。ARIMAモデルには、通常、$p$、$d$、$q$と呼ばれる3つの主要なパラメーターがあります。

- **p**：ARIMAの自己回帰部分の次数
- **d**：関係する差分の程度
- **q**：移動平均部分の次数

これらは通常、ARIMA(p, d, q)の順序で書かれます。多くのプログラミング言語やパッケージは、分析する時系列とこれら3つのパラメータを使用して呼び出すことができるARIMA関数を提供しています。ほとんどの場合、データはトレーニング・セットとテスト・セットに分割され、トレーニング後にモデルの精度をテストできます。通常、時間プロットを見だけで、データに最も適切なpとqの値を知ることはできません。ただし、ACFおよびPACFプロットを使用してpとqの適切な値を決定できることが多いため、これらのプロットはARIMAを扱う上で重要な用語です。

モデルでAR用語を使用する大まかな基準は次のとおりです。

- ACFプロットは自己相関がゼロに向かって減少していることを示す
- PACFプロットはゼロに向かって急速に途切れる
- 定常系列のACFはLag - 1で正を示す

モデルでMAの用語を使用する状況のおおよその基準は、次の場合です。

- ラグ-1で負に自己相関
- 数回の遅れで急激に低下するACF
- PACFが突然ではなく徐々に減少

遭遇する可能性のある従来のARIMAモデル・タイプがいくつかあります。

- **ARIMA (1,0,0) = 1次自己回帰モデル**: 系列が定常で自己相関関係にある場合、おそらくそれ自体の以前の値の倍数に定数を加えたものとして予測できます。今日からのアイスクリームの売上だけを使って明日のアイスクリームの売上を直接予測できるとしたら、それは一次自己回帰モデルです。
- **ARIMA(0,1,0) = ランダム・ウォーク**: 時系列が定常でない場合、最も単純なモデルはランダム・ウォーク・モデルです。ランダム・ウォークは、シーケンス内の次の値がシーケンス内の前の値の修正であるため、乱数のリストとは異なります。これは、よくある、株価の差異値のモデル化方法です。
- **ARIMA(1,1,0) = 差分一次自己回帰モデル**: ランダム・ウォーク・モデルのエラーが自己相関している場合、予測方程式に従属変数の1つのラグを追加することで、つまり、Yの最初の差分を1期間ラグしたそれ自体に回帰することで、問題を解決できる可能性があります。
- **定数なしのARIMA(0,1,1) = 単純な指数平滑化モデル**: これは、季節性やトレンドのない時系列データに使用されます。過去の観測からの影響度（0～1の間の係数値で示される）を制御する単一の平滑化パラメーターが必要です。この手法では、1に近い値はモデルが過去の観測値にほとんど注意を払うことを意味し、小さい値は予測中により多くの履歴を考慮することを意味します。
- **定数付きARIMA(0,1,1) = 単純な指数平滑化成長モデル**: これは、単純な指数平滑化と同じですが、時系列のY値が進行するにつれて増大する付加的な定数項がある点が異なります。

もちろん、ARIMAモデルを適合させる方法は他にもたくさんあります。そのため、複数のモデルを計算し、比較して、どのモデルがデータに最もよく適合するかを確認することがよくあります。これらはすべて一次モデルであり、線形プロセスをマッピングすることを意味します。二次プロセスをマッピングする2次モデルと、より複雑なプロセスをマッピングする上位モデルがあります。

## ARIMAモデルの比較
通常、複数のARIMAモデルをデータに適合させ、相互に比較することで、どの資産が時系列データに見られるパターンを予測しているかを見つけられます。ARIMAモデルの精度を評価するための重要な指標は次の3つです。

1. **Akaikeの情報基準またはAIC**。これは、回帰モデルの予測子の選択に広く使用されており、ARIMAモデルの次数の決定にも役立ちます。AICは、モデルの適合性とモデルの単純さないし倹約性の両方を単一の統計情報で定量化します。AICスコアが低いほど良いため、スコアが低いモデルを好みます。AICはより単純なモデルを優先し、より複雑なモデルは、その精度がより単純なモデルとほぼ同じである限り、より高いスコアを獲得します。また、単にサンプル・サイズに小さな修正を加えた修正済みAICやAICCもあります。
2. **ベイズ情報基準（BIC）**。これは、AICよりも多くの複雑さにペナルティーを与えるモデル選択のもう1つの基準です。AICと同様に、BICの低いモデルは、一般的にスコアの高いモデルよりも優先されます。モデルが長期予測に使用される場合は、BICの方が適している可能性がありますが、短期予測にはAICの方が適しているということかもしれません。
3. **シグマ二乗またはsigma2値**は、モデル残差の分散です。シグマは、仮想プロセスの不安定性を表します。揮発性が高いデータにもかかわらずシグマ二乗スコアが非常に低い場合や、逆に非揮発性データであるがシグマ二乗スコアが高い場合は、モデルが実際のデータ生成プロセスをうまく捉えていないことを示しています。

テスト・データ・セットを留保した場合は、異なる予測間隔のRMSEなどの精度メトリクスを比較することもできます。ARIMAモデルは、将来の単一の時間ステップまたは一度に複数のステップの値を予測できます。

## ARIMAのバリエーション
ARIMAモデルの構成と比較に対するもう1つのアプローチは、自動化された構成タスクをARIMAモデルの生成と比較に適用するAuto-ARIMAを使用することです。最適なモデルを得るには、複数の方法があります。アルゴリズムは複数のモデルを生成し、AICcと最尤推定の誤差を最小化してARIMAモデルを得ようとします。

**季節性自己回帰統合移動平均、SARIMAまたは季節性ARIMA**は、季節的要素を含む時系列データをサポートするARIMAの拡張版です。そのために、系列の季節成分の自己回帰、差分、移動平均を指定する3つの新しいハイパーパラメーターと、季節性の期間のパラメーターを追加します。SARIMAモデルは通常、SARIMA((p,d,q),(P,D,Q))と表現され、小文字は時系列の非季節性成分を示し、大文字は季節性成分を示します。

**ベクトル自己回帰モデル（またはVARモデル）**は、多変量の時系列に使用されます。これらは、各変数がそれ自体の過去の遅延と他の変数の過去の遅延の線形関数になるように構成されています。

ARIMAモデルは、時系列データを分析して過去のプロセスを理解したり、時系列の将来の値を予測したりするための強力なツールです。ARIMAモデルは、自己回帰モデルと移動平均モデルを組み合わせて、予測者にさまざまな時系列データを使用できる高度にパラメーター化可能なツールを提供します。
